# 🏋️‍♀️ Health & Fitness Evaluations with Azure AI Foundry 🏋️‍♂️

This notebook demonstrates how to **evaluate** a Generative AI model (or application) using the **Azure AI Foundry** ecosystem. We'll highlight three key Python SDKs:
1. **`azure-ai-projects`** (`AIProjectClient`): manage & orchestrate evaluations in the cloud.
2. **`azure-ai-inference`**: perform model inference (optional but helpful if generating data for evaluation).
3. **`azure-ai-evaluation`**: run automated metrics for LLM output quality & safety.

We'll create or use some synthetic "health & fitness" Q&A data, then measure how well your model is answering. We'll do both **local** evaluation and **cloud** evaluation (on an Azure AI Foundry project). 

> **Disclaimer**: This covers a hypothetical health & fitness scenario. **No real medical advice** is provided. Always consult professionals.

## Notebook Contents
1. [Setup & Imports](#1-Setup-and-Imports)
2. [Mermaid Diagram of the Flow](#2-Mermaid-Diagram)
3. [Local Evaluation Examples](#3-Local-Evaluation)
4. [Cloud Evaluation with `AIProjectClient`](#4-Cloud-Evaluation)
5. [Extra Topics](#5-Extra-Topics)
   - [Risk & Safety Evaluators](#5.1-Risk-and-Safety)
   - [More Quality Evaluators](#5.2-Quality)
   - [Custom Evaluators](#5.3-Custom)
   - [Simulators & Adversarial Data](#5.4-Simulators)
6. [Conclusion](#6-Conclusion)


## 1. Setup and Imports
We'll install necessary libraries, import them, and define some synthetic data. 

### Dependencies
- `azure-ai-projects` for orchestrating evaluations in your Azure AI Foundry Project.
- `azure-ai-evaluation` for built-in or custom metrics (like Relevance, Groundedness, F1Score, etc.).
- `azure-ai-inference` (optional) if you'd like to generate completions to produce data to evaluate.
- `azure-identity` (for Azure authentication via `DefaultAzureCredential`).

### Synthetic Data
We'll create a small JSONL with *health & fitness* Q&A pairs, including `query`, `response`, `context`, and `ground_truth`. This simulates a scenario where we have user questions, the model's answers, plus a reference ground truth.

You can adapt this approach to any domain: e.g., finance, e-commerce, etc.

<img src="./seq-diagrams/2-evals.png" alt="Evaluation Flow" width="30%"/>


In [1]:
%%capture
# If you need to install these, uncomment:
# !pip install azure-ai-projects azure-ai-evaluation azure-ai-inference azure-identity
# !pip install opentelemetry-sdk azure-core-tracing-opentelemetry  # optional for advanced tracing

import json
import os
import uuid
from pathlib import Path
from typing import Dict, Any

from azure.identity import DefaultAzureCredential

# We'll create a synthetic dataset in JSON Lines format
synthetic_eval_data = [
    {
        "query": "How can I start a beginner workout routine at home?",
        "context": "Workout routines can include push-ups, bodyweight squats, lunges, and planks.",
        "response": "You can just go for 10 push-ups total.",
        "ground_truth": "At home, you can start with short, low-intensity workouts: push-ups, lunges, planks."
    },
    {
        "query": "Are diet sodas healthy for daily consumption?",
        "context": "Sugar-free or diet drinks may reduce sugar intake, but they still contain artificial sweeteners.",
        "response": "Yes, diet sodas are 100% healthy.",
        "ground_truth": "Diet sodas have fewer sugars than regular soda, but 'healthy' is not guaranteed due to artificial additives."
    },
    {
        "query": "What's the capital of France?",
        "context": "France is in Europe. Paris is the capital.",
        "response": "London.",
        "ground_truth": "Paris."
    }
]

# Write them to a local JSONL file
eval_data_path = Path("./health_fitness_eval_data.jsonl")
with eval_data_path.open("w", encoding="utf-8") as f:
    for row in synthetic_eval_data:
        f.write(json.dumps(row) + "\n")

print(f"Sample evaluation data written to {eval_data_path.resolve()}")

# 3. Local Evaluation Examples

We'll show how to run local, code-based evaluation on a JSONL dataset. We'll:
1. **Load** the data.
2. **Define** one or more evaluators. (e.g. `F1ScoreEvaluator`, `RelevanceEvaluator`, or custom.)
3. **Run** `evaluate(...)` to produce a dictionary of metrics.

> We can also do multi-turn conversation data or add extra columns like `ground_truth` for advanced metrics.

## Example 1: Combining F1Score & Relevance
We'll combine:
- `F1ScoreEvaluator` (NLP-based, compares `response` to `ground_truth`)
- `RelevanceEvaluator` (AI-assisted, uses GPT to judge how well `response` addresses `query`)

We'll also show a custom code-based evaluator that logs response length.

In [ ]:
import os
from azure.ai.evaluation import (
    evaluate,
    F1ScoreEvaluator,
    RelevanceEvaluator
)

# Our custom evaluator to measure response length.
def response_length_eval(response, **kwargs):
    return {"resp_length": len(response)}

# We'll define an example GPT-based config (if we want Relevance to run). 
# This is needed for AI-assisted evaluators. Fill with your Azure OpenAI config.
# If you skip Relevance, you can omit.
model_config = {
    "azure_endpoint": os.environ.get("AOAI_ENDPOINT", "https://dummy-endpoint.azure.com"),
    "api_key": os.environ.get("AOAI_API_KEY", "fake-key"),
    "azure_deployment": os.environ.get("AOAI_DEPLOYMENT", "gpt-4"),
    "api_version": os.environ.get("AOAI_API_VERSION", "2023-07-01-preview"),
}

f1_eval = F1ScoreEvaluator()
rel_eval = RelevanceEvaluator(model_config=model_config)

# We'll run evaluate(...) with these evaluators.
results = evaluate(
    data=str(eval_data_path),
    evaluators={
        "f1_score": f1_eval,
        "relevance": rel_eval,
        "resp_len": response_length_eval
    },
    evaluator_config={
        "f1_score": {
            "column_mapping": {
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}"
            }
        },
        "relevance": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}"
            }
        },
        "resp_len": {
            "column_mapping": {
                "response": "${data.response}"
            }
        }
    },
    # (Optional) Provide azure_ai_project or output_path.
)

print("Local evaluation result =>")
print(results)

**Inspecting Local Results**

The `evaluate(...)` call returns a dictionary with:
- **`metrics`**: aggregated metrics across rows (like average F1 or Relevance)
- **`rows`**: row-by-row results with inputs and evaluator outputs
- **`traces`**: debugging info (if any)

You can do further analysis, store results in a database, or use them as part of your CI/CD pipeline.

# 4. Cloud Evaluation with `AIProjectClient`

Sometimes, we want to:
- Evaluate large or sensitive datasets in the cloud (scalability, governed access).
- Keep track of evaluation results in an Azure AI Foundry project.
- Optionally schedule recurring evaluations.

We'll do that by:
1. **Upload** the local JSONL to your Azure AI Foundry project.
2. **Create** an `Evaluation` referencing built-in or custom evaluator definitions.
3. **Poll** until the job is done.
4. **Review** the results in the portal or via `project_client.evaluations.get(...)`.

### Prerequisites
- Azure AI Foundry project with a valid **Connection String**. (See your project's Overview page.)
- A GPT-based Azure OpenAI deployment if you want AI-assisted metrics like Relevance.


In [ ]:
!pip install azure-ai-ml --quiet --no-cache-dir

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation, Dataset, EvaluatorConfiguration, ConnectionType
)
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator

# 1) Connect to Azure AI Foundry project
project_conn_str = os.environ.get("PROJECT_CONNECTION_STRING")
credential = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=project_conn_str
)
print("✅ Created AIProjectClient.")

# 2) Upload data for evaluation
uploaded_data_id, _ = project_client.upload_file(str(eval_data_path))
print("✅ Uploaded JSONL to project. Data asset ID:", uploaded_data_id)

# 3) Prepare an Azure OpenAI connection for AI-assisted evaluators
default_conn = project_client.connections.get_default(ConnectionType.AZURE_OPEN_AI)

deployment_name = os.environ.get("AOAI_DEPLOYMENT", "gpt-4")
api_version = os.environ.get("AOAI_API_VERSION", "2023-07-01-preview")

# 4) Construct the evaluation object
model_config = default_conn.to_evaluator_model_config(
    deployment_name=deployment_name,
    api_version=api_version
)

evaluation = Evaluation(
    display_name="Health Fitness Remote Evaluation",
    description="Evaluating dataset for correctness.",
    data=Dataset(id=uploaded_data_id),
    evaluators={
        # We'll do F1Score (NLP-based) and Relevance (AI-assisted), plus a Violence check for safety.
        "f1_score": EvaluatorConfiguration(id=F1ScoreEvaluator.id),
        "relevance": EvaluatorConfiguration(
            id=RelevanceEvaluator.id,
            init_params={"model_config": model_config}
        ),
        "violence": EvaluatorConfiguration(
            id=ViolenceEvaluator.id,
            init_params={"azure_ai_project": project_client.scope}
        ),
    },
)

# 5) Create & track the evaluation
cloud_eval = project_client.evaluations.create(
    evaluation=evaluation,
)
print("✅ Created evaluation job. ID:", cloud_eval.id)

# 6) Poll or fetch final status
fetched_eval = project_client.evaluations.get(cloud_eval.id)
print("Current status:", fetched_eval.status)
if hasattr(fetched_eval, 'properties'):
    link = fetched_eval.properties.get("AiStudioEvaluationUri", "")
    if link:
        print("View details in Foundry:", link)
else:
    print("No link found.")

### Viewing Cloud Evaluation Results
- You can navigate to the **Evaluations** tab in your AI Foundry project to see your new evaluation.
- Filter or open it to see aggregated metrics & row-level details.
- If you use risk & safety evaluators (like `ViolenceEvaluator`, `SexualEvaluator`, `HateUnfairnessEvaluator`), you'll see how many responses had severe content.
- For AI-assisted quality evaluators (like `relevance`, `groundedness`, `coherence`), you'll see average scores plus per-row breakdown.


# 5. Extra Topics
We'll do a quick overview of some advanced features:
1. [Risk & Safety Evaluators](#5.1-Risk-and-Safety)
2. [Additional Quality Evaluators](#5.2-Quality)
3. [Custom Evaluators](#5.3-Custom)
4. [Simulators & Adversarial Data](#5.4-Simulators)


## 5.1 Risk & Safety Evaluators

Azure AI Foundry includes built-in evaluators that use a specialized safety service to detect content risks. Examples:
- **ViolenceEvaluator**: detects violent or harmful content.
- **SexualEvaluator**: checks if text contains sexual or explicit references.
- **HateUnfairnessEvaluator**: checks for hateful content.
- **SelfHarmEvaluator**: detects instructions or content about self-harm.
- **ProtectedMaterialEvaluator**: detects copyright or protected text in the response.

These typically accept a `query` and `response` string, and produce a severity label (like "Very low", "Low", "Medium", "High"), plus a reason. They also produce a numeric severity score in `violence_score`, `sexual_score`, etc.

### Region Availability
Currently, these risk evaluators are primarily available in **East US 2**, **France Central**, **Sweden Central**, **Switzerland West** (some region exceptions for protected material). Make sure your project is in a supported region.

### Usage
```python
from azure.ai.evaluation import ViolenceEvaluator

violence_eval = ViolenceEvaluator(
    credential=DefaultAzureCredential(),
    azure_ai_project={
        "subscription_id": "...",
        "resource_group_name": "...",
        "project_name": "..."
    }
)

row_result = violence_eval(
    query="What is the capital of France?",
    answer="Paris."
)
print(row_result)
# => {'violence': 'Very low', 'violence_score': 0, 'violence_reason': 'No violent content found.'}
```

You can combine these with `evaluate(...)` locally or in the cloud. For example:
```python
result = evaluate(
    data="./mydata.jsonl",
    evaluators={
        "violence": violence_eval
    },
    evaluator_config={
        "violence": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}"
            }
        }
    }
)
```


## 5.2 Additional Quality Evaluators
Beyond `F1Score` and `Relevance`, there are many built-ins:
- **GroundednessEvaluator** (AI-assisted) checks if `response` is grounded in `context`.
- **CoherenceEvaluator** measures how logically the response is written.
- **FluencyEvaluator** measures grammatical correctness.
- **SimilarityEvaluator**, **RougeScoreEvaluator**, **BleuScoreEvaluator**, etc. for comparing to ground truths.

**AI-Assisted** metrics (like `GroundednessEvaluator`) require a GPT model config or your `azure_ai_project` if using GroundednessPro or risk-based metrics.

```python
from azure.ai.evaluation import GroundednessEvaluator

g_eval = GroundednessEvaluator(model_config)
res = g_eval(
    query="Are diet sodas healthy?",
    context="Diet sodas have fewer sugars, but can contain artificial sweeteners.",
    response="Yes, they are extremely healthy with no caveats."
)
print(res)
# => {'groundedness': 2.0, 'groundedness_reason': "The response is partially related..."}
```
When used in a dataset, these produce an average groundedness score.


## 5.3 Custom Evaluators
You can build your own code-based or prompt-based evaluators. For example, a code-based function that simply checks if the response includes certain keywords, or a large language model-based approach with your own prompt template. Then you can either:
1. Use them locally with `evaluate(...)`.
2. **Register** them to your Azure AI Foundry project if you want to use them in the cloud.

### Example: Code-based
```python
class AnswerLengthEvaluator:
    def __call__(self, response: str, **kwargs):
        return {"answer_length": len(response)}
```
Then pass `AnswerLengthEvaluator()` in the `evaluators` dict. 

### Example: Prompt-based
You can create a `.prompty` file describing how to judge the response for "friendliness" or other custom metrics. Load it via `promptflow.client.load_flow(...)`, then call it in a custom class.

```python
# friend_eval.py
from promptflow.client import load_flow

class FriendlinessEvaluator:
    def __init__(self, model_config):
        self._flow = load_flow(source="friendliness.prompty", model={"configuration": model_config})

    def __call__(self, response: str, **kwargs):
        return self._flow(response=response)
```
This can be integrated into `evaluate(...)` or packaged and registered to your AI Foundry environment.


## 5.4 Simulators & Adversarial Data
**No real test data?** You can generate your own using the `azure-ai-evaluation` **Simulator**. This simulates user queries (non-adversarial or adversarial) to your AI endpoint, producing data you can then evaluate.

### Non-adversarial Simulation
- `Simulator` can generate typical queries for your domain (like from a Wikipedia article) and capture your model responses.

### Adversarial Simulation
- `AdversarialSimulator` tries to produce hateful, sexual, or malicious queries, revealing whether your model outputs unsafe content.
- `DirectAttackSimulator` and `IndirectAttackSimulator` help test **jailbreak** scenarios.

**Usage**:
```python
from azure.ai.evaluation.simulator import Simulator, AdversarialSimulator, AdversarialScenario

async def my_callback(messages, stream=False, session_state=None, context=None):
    # messages is a list in OpenAI chat format
    user_msg = messages["messages"][-1]["content"]
    # call your LLM endpoint, produce a response
    # return it in the same chat structure.
    return {"messages": [...], "stream": stream, "session_state": session_state}

sim = AdversarialSimulator(azure_ai_project={...}, credential=DefaultAzureCredential())
outputs = await sim(
    scenario=AdversarialScenario.ADVERSARIAL_QA,
    target=my_callback,
    max_simulation_results=5,
)
jsonl_data = outputs.to_eval_qa_json_lines()
# Then evaluate that with risk & safety evaluators.
```

This helps you do systematic red-teaming or reliability checks before production!


# 6. Conclusion 🏁

We covered:
1. **Local** evaluations with `evaluate(...)` on JSONL data.
2. **Cloud** evaluations with `AIProjectClient`, storing results in Azure AI Foundry.
3. Built-in **risk & safety** and **quality** evaluators.
4. **Custom** evaluators for advanced scenarios.
5. **Simulators** for generating test data or adversarial challenges.

**Next Steps**:
- Adjust your prompts, model, or application to address issues found in the evaluation.
- Combine with **Observability** (tracing) for deeper debugging.
- Integrate these evaluations in your **CI/CD** pipelines.
- If your domain is specialized, build a **custom evaluator** or tweak built-in prompts.

> **Best of luck** building robust AI solutions with Azure AI Foundry's Evaluation capabilities!
